## Importa funções para o pré-processamento

In [1]:
from pre_processamento import *

from dlisio import dlis
import pandas as pd
import numpy as np

# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

## Carrega os dados

In [2]:
nomes_arquivos = []     # Armazena os nomes dos arquivos .dlis
leituras_dlis = []      # Armazena as leituras dos arquivos .dlis
nomes_anp = []          # Armazena os nomes obtidos das leituras

for file in glob.glob(r'**/Data' + "/*.dlis", recursive=True):
    # Salva o nome do arquivo
    nomes_arquivos.append(file)

    # Salva os dados da leitura
    leitura, *tail = dlis.load(f'{file}')
    leituras_dlis.append(leitura)

    # Salva o nome do poço
    nome = leitura.origins[0].well_name
    nomes_anp.append(nome)

In [3]:
print(nomes_arquivos)
print(nomes_anp)

['Data\\1-brsa-551-se_brsa_raw.dlis', 'Data\\1-brsa-574-se_brsa_raw.dlis', 'Data\\1-brsa-595-se_brsa_raw-1.dlis', 'Data\\1-brsa-605-se_brsa_raw.dlis', 'Data\\1-brsa-659-se_brsa_raw.dlis', 'Data\\3-CP-1847-SE.dlis', 'Data\\3-CP-1848-SE.dlis', 'Data\\3-CP-1851-SE.dlis', 'Data\\3-CP-1853-SE.dlis', 'Data\\3-CP-1855-SE.dlis', 'Data\\3-CP-1857-SE.dlis']
['1-FSG-1-SE', '1-FSJQ-1-SE', '1-BRSA-595-SE', '1-BRSA-605-SE', '1-BRSA-659-SE', '3-BRSA-900-SE', '3-BRSA-889-SE', '3-BRSA-912-SE', '3-BRSA-897-SE', '3-BRSA-910-SE', '3-BRSA-907-SE']


## Padroniza nomes fora do padão 'X-BRSA-XXX-SE'

In [4]:
nomes_anp[0] = '1-BRSA-551-SE'
nomes_anp[1] = '1-BRSA-574-SE'

In [5]:
nomes_anp

['1-BRSA-551-SE',
 '1-BRSA-574-SE',
 '1-BRSA-595-SE',
 '1-BRSA-605-SE',
 '1-BRSA-659-SE',
 '3-BRSA-900-SE',
 '3-BRSA-889-SE',
 '3-BRSA-912-SE',
 '3-BRSA-897-SE',
 '3-BRSA-910-SE',
 '3-BRSA-907-SE']

## Cria dicionário para armazenar os dados e respectivos nomes

In [7]:
# Casa itens da lista 'nome_anp_abreviados' com os itens da lista 'leituras_dlis'
pares = zip(nomes_anp, leituras_dlis)

# Cria dicionário 'dli_dict'
dli_dict = dict(pares)
dli_dict

{'1-BRSA-551-SE': LogicalFile(AIT_SONIC_TLD_MCFL_018PUP),
 '1-BRSA-574-SE': LogicalFile(GEOLOAD.1),
 '1-BRSA-595-SE': LogicalFile(GEOLOAD.1),
 '1-BRSA-605-SE': LogicalFile(AIT_SONIC_TLD_MCFL_048PUC),
 '1-BRSA-659-SE': LogicalFile(GEOLOAD.1),
 '3-BRSA-900-SE': LogicalFile(GEOLOAD.1),
 '3-BRSA-889-SE': LogicalFile(GEOLOAD.1),
 '3-BRSA-912-SE': LogicalFile(GEOLOAD.1),
 '3-BRSA-897-SE': LogicalFile(GEOLOAD.1),
 '3-BRSA-910-SE': LogicalFile(GEOLOAD.1),
 '3-BRSA-907-SE': LogicalFile(GEOLOAD.1)}

## Separa TODAS as curvas presentes nos .dlis de cada poço

In [8]:
channels_dict = {}

for key, poco in dli_dict.items():
    channels_list = []
    for frame in poco.frames:
        channels = frame.channels
        channels_list.append([channel.name for channel in channels])
    channels_dict[key] = sum(channels_list, [])

## Salva as curvas presentes nos .dlis em arquivos CSV

In [9]:
import csv

for key, value in channels_dict.items():
    file_name = f"Curvas_CSV/curvas_{key}.csv"
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Curva", "Poco"])
        for item in value:
            writer.writerow([item, f'{key}'])

    print(f"Arquivo {file_name} criado com sucesso")

Arquivo Curvas_CSV/curvas_1-BRSA-551-SE.csv criado com sucesso
Arquivo Curvas_CSV/curvas_1-BRSA-574-SE.csv criado com sucesso
Arquivo Curvas_CSV/curvas_1-BRSA-595-SE.csv criado com sucesso
Arquivo Curvas_CSV/curvas_1-BRSA-605-SE.csv criado com sucesso
Arquivo Curvas_CSV/curvas_1-BRSA-659-SE.csv criado com sucesso
Arquivo Curvas_CSV/curvas_3-BRSA-900-SE.csv criado com sucesso
Arquivo Curvas_CSV/curvas_3-BRSA-889-SE.csv criado com sucesso
Arquivo Curvas_CSV/curvas_3-BRSA-912-SE.csv criado com sucesso
Arquivo Curvas_CSV/curvas_3-BRSA-897-SE.csv criado com sucesso
Arquivo Curvas_CSV/curvas_3-BRSA-910-SE.csv criado com sucesso
Arquivo Curvas_CSV/curvas_3-BRSA-907-SE.csv criado com sucesso


## Concatena os arquivos CSV das curvas em um único arquivo

In [10]:
import os

pasta = "Curvas_CSV"

# Lista para armazenar os dataframes repectivos aos .csv
df_list = []

# Itera sobre a pasta Curvas_CSV
for arquivo in os.listdir(pasta):
    file_path = os.path.join(pasta, arquivo)

    # Lê o arquivo CSV como um DF
    df = pd.read_csv(file_path)
    df_list.append(df)

# Concatena os DFs ao longo do eixo das colunas
df_concat = pd.concat(df_list, axis=0, ignore_index=True)

# Salva o DF concatenado em um .csv
df_concat.to_csv("curvas_pocos.csv", index=False)

print("Arquivo curvas_pocos.csv criado com sucesso")

Arquivo curvas_pocos.csv criado com sucesso


## Cria dataframes para os poços

Alguns poços possuem mais de um 'frame'.

Irei criar um 'dataframe' para cada um desses 'frames'.

Irei fazer um 'JOIN' usando a coluna 'TDEP'.

### Teste 1

In [59]:
# Iniciando o teste com um poço apenas
poco = dli_dict['1-BRSA-605-SE']

# Dicionário para armazenar os dataframes dos frames do poço 605
dataframes_605 = {}

# Iterando sobre os frames do poço
for frame in poco.frames:
    # indice do frame
    indice = poco.frames.index(frame)
    
    # obtendo os valores das curvas
    curvas = frame.curves()

    dataframes_605[indice] = pd.DataFrame(columns=curvas)


In [60]:
dataframes_605[0]

,"(1, 264240.0, 8.5, 2009.0, 135.375, 47.26148986816406, 0.26395753026008606, 0.0, 0.0, 0.0, 82.79398345947266)","(2, 264180.0, 8.5, 1731.0, 136.3125, 47.26148986816406, 0.26395753026008606, 2184.0, 0.0, 0.0, 82.79398345947266)","(3, 264120.0, 8.5, 1811.0, 135.75, 47.26148986816406, 0.26395753026008606, 1879.0, 0.0, 0.0, 82.79398345947266)","(4, 264060.0, 8.5, 1776.0, 136.25, 47.26148986816406, 0.26395753026008606, 1297.5, 0.0, 0.0, 82.79398345947266)","(5, 264000.0, 8.5, 1871.0, 135.375, 47.26148986816406, 0.26395753026008606, 1131.0, 0.0, 0.0, 82.79398345947266)","(6, 263940.0, 8.5, 1896.0, 134.9375, 47.26148986816406, 0.26395753026008606, 1095.0, 0.0, 0.0, 82.79398345947266)","(7, 263880.0, 8.5, 1979.0, 136.3125, 47.26148986816406, 0.26395753026008606, 1023.5, 0.0, 0.0, 82.79398345947266)","(8, 263820.0, 8.5, 2042.0, 135.3125, 47.26148986816406, 0.26395753026008606, 941.0, 0.0, 0.0, 82.79398345947266)","(9, 263760.0, 8.5, 1834.0, 135.4375, 47.26148986816406, 0.26395753026008606, 824.5, 0.0, 0.0, 82.79398345947266)","(10, 263700.0, 8.5, 1794.0, 135.4375, 47.26148986816406, 0.26395753026008606, 803.0, 0.0, 0.0, 82.79398345947266)",...,"(4231, 10440.0, 8.5, 1132.0, 204.5, 62.600650787353516, 0.5468772053718567, 823.5, 16.10105323791504, 5.987124443054199, 60.0)","(4232, 10380.0, 8.5, 1117.0, 204.5, 64.2231674194336, 0.5584238767623901, 824.5, 16.10105323791504, 5.987124443054199, 60.0)","(4233, 10320.0, 8.5, 1127.0, 204.125, 63.33281326293945, 0.5747538805007935, 805.5, 16.10105323791504, 5.987124443054199, 60.0)","(4234, 10260.0, 8.5, 1118.0, 204.4375, 63.19962692260742, 0.5424395203590393, 804.0, 16.10105323791504, 5.987124443054199, 60.0)","(4235, 10200.0, 8.5, 1129.0, 204.125, 59.82231521606445, 0.4838324189186096, 813.5, 16.10105323791504, 5.987124443054199, 60.0)","(4236, 10140.0, 8.5, 1128.0, 204.375, 59.68840026855469, 0.4633655846118927, 808.5, 16.10105323791504, 5.987124443054199, 60.0)","(4237, 10080.0, 8.5, 1136.0, 204.5625, 59.50723648071289, 0.4595179557800293, 825.5, 16.10105323791504, 5.987124443054199, 60.0)","(4238, 10020.0, 8.5, 1149.0, 204.1875, 59.372772216796875, 0.4608493149280548, 810.5, 16.10105323791504, 5.987124443054199, 60.0)","(4239, 9960.0, 8.5, 1162.0, 204.875, 60.82278060913086, 0.48407942056655884, 824.5, 16.10105323791504, 5.987124443054199, 60.0)","(4240, 9900.0, 8.5, 1181.0, 204.875, 60.82278060913086, 0.48407942056655884, 808.5, 16.10105323791504, 5.987124443054199, 60.0)"


In [54]:
poco = dli_dict['1-BRSA-605-SE']
data = poco.frames[2].curves()
data

array([(   1, 264240., 1234.8523 , 0.7907106 ,   0.  , 0.8127777 ),
       (   2, 264210., 1234.7153 , 0.79074144,   0.  , 0.81280535),
       (   3, 264180., 1234.0337 , 0.79075444,   0.  , 0.8127714 ), ...,
       (8476,   9990.,  156.57849, 7.8798385 , 412.25, 7.0442214 ),
       (8477,   9960.,  156.57849, 7.8798385 , 412.25, 7.0442214 ),
       (8478,   9930.,  156.57849, 7.8798385 , 404.25, 7.0442214 )],
      dtype=[('FRAMENO', '<i4'), (('T.CHANNEL-I.TDEP-O.10-C.3', 'TDEP'), '<f4'), (('T.CHANNEL-I.AHTCO90-O.10-C.0', 'AHTCO90'), '<f4'), (('T.CHANNEL-I.AHF30-O.10-C.0', 'AHF30'), '<f4'), (('T.CHANNEL-I.TIME-O.10-C.3', 'TIME'), '<f4'), (('T.CHANNEL-I.AHF90-O.10-C.0', 'AHF90'), '<f4')])

In [66]:
dlis_df_dict = {}   # Conterá os dataframes respectivos aos poços

# Curvas de perfis escolhidas
curvas_escolhidas = ['TDEP', 'GR', 'NPHI', 'RHOB', 'DRHO', 'HDRS', 'LLD', 'BSZ', 'BS', 'CALI', 'DCALI', 'DCAL', 'PE', 'DTC']

# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for chave, poco in dli_dict.items():

    # Armazenando as curvas que serão utilizadas em uma lista
    curvas_utilizadas = [
        channel.name                                    # Os elementos da lista serão os nomes das curvas
        for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
        if channel.name in curvas_escolhidas            # As curvas que não utilizaremos não serão armazenadas na lista
    ]
    conjunto_aux = set(curvas_utilizadas)
    curvas_utilizadas_sem_duplicados = list(conjunto_aux)
    
    #frames = [np.atleast_1d(frame) for frame in poco.frames]
    #curvas = np.concatenate(frames, axis=0)
    dataframe = pd.DataFrame()
    
    for frame in poco.frames:

        curvas = frame.curves()

        dataframe = pd.concat([dataframe, pd.DataFrame(curvas[curvas_utilizadas_sem_duplicados])])

    # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
    dlis_df_dict[chave] = dataframe


KeyError: 'GR'

In [64]:
dlis_df_dict.keys()

dict_keys(['551', '574', '595', '605', '659', '900', '889', '912', '897', '910', '907'])

## Transforma os valores -999.25 em nulos

In [ ]:
for poco in dlis_df_dict.values():
    poco.replace([-999.25], [None], inplace = True)

## Aplicando os mnemônicos

In [ ]:
aplica_mnemonico(dlis_df_dict, ['BS', 'BSZ'], 'BS')
aplica_mnemonico(dlis_df_dict, ['LLD', 'HDRS'], 'RESD')
aplica_mnemonico(dlis_df_dict, ['RHOB', 'RHOZ'], 'RHOB')

## Renomeia CALI para CAL

In [ ]:
for poco in dlis_df_dict.values():
    renomeia_coluna(poco, 'CALI', 'CAL')

## Adiciona coluna DCAL

In [ ]:
add_DCAL(dlis_df_dict)

## Preenche os poços com curvas faltando

In [ ]:
# Se um dos poços não tiver uma dessas curvas, adicionamos a coluna da curva e mantemos os valores como None
curvas_obrigatorias = ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']

# Percorre todos os poços
for poco in dlis_df_dict.values():
    # Percorre todas as curvas obrigatórias
    for curva in curvas_obrigatorias:
        # Se o poço não tiver a curva
        if curva not in poco.columns:
            # Adiciona a coluna e os valores dela = None
            poco[curva] = None

## Temos as seguintes curvas 

In [ ]:
for key, poco in dlis_df_dict.items():
    curvas = sorted(poco.keys())
    print(f"{key}: {curvas}")

551: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
574: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
595: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
605: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
659: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
900: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
889: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
912: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
897: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
910: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
907: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']


## Remove valores DRHO e DCAL indesejados (Só depois)

In [ ]:
#limita_curva(dlis_df_dict, "DRHO", -0.15, 0.15)
#limita_curva(dlis_df_dict, "DCAL", -2, 2)

## Inverte a ordem das linhas dos dataframes

In [ ]:
for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].iloc[::-1]

## Ordena os dataframes

In [ ]:
ordem_desejada = ['TDEP', 'GR', 'RESD', 'BS', 'CAL', 'DCAL', 'NPHI', 'PE', 'DRHO', 'RHOB', 'DTC']

for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].reindex(columns=ordem_desejada)

## Salva os dados dos dataframes em arquivos CSV

In [ ]:
for key, value in dlis_df_dict.items():
    file_name = f"Pocos_CSV/poco_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

Arquivo Pocos_CSV/poco_551.csv criado com sucesso.
Arquivo Pocos_CSV/poco_574.csv criado com sucesso.
Arquivo Pocos_CSV/poco_595.csv criado com sucesso.
Arquivo Pocos_CSV/poco_605.csv criado com sucesso.
Arquivo Pocos_CSV/poco_659.csv criado com sucesso.
Arquivo Pocos_CSV/poco_900.csv criado com sucesso.
Arquivo Pocos_CSV/poco_889.csv criado com sucesso.
Arquivo Pocos_CSV/poco_912.csv criado com sucesso.
Arquivo Pocos_CSV/poco_897.csv criado com sucesso.
Arquivo Pocos_CSV/poco_910.csv criado com sucesso.
Arquivo Pocos_CSV/poco_907.csv criado com sucesso.
